<a href="https://colab.research.google.com/github/MadScientist85/CodingIT/blob/main/Agents/Conditional_Router_Agent_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Router Agent Workflow - Selecting the right LLM for the job
Author: [Zain Hasan](https://x.com/ZainHasan6)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Agents/Conditional_Router_Agent_Workflow.ipynb)

## Introduction

In this notebook, we'll demonstrate how to program an agent workflow that dynamically selects LLMs based on their specialties and task requirements.

For example, coding tasks might use specialized models like Qwen coder or DeepSeek 2.5, while planning or reasoning tasks could leverage different models optimized for those purposes.

To achieve this, we'll create:

1. LLM Router: A language model that selects and justifies the best model for a given task
2. Simple API that executes the chosen model and solves the task

## Conditional Router Agent Workflow

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/if_router.png?raw=1" width="700">

In this **conditional router agent workflow**, we demonstrate how to program a router LLM that selects from multiple routes - including different LLMs, prompts, action sequences, or functionalities. Given a task and route preferences, it chooses one path and justifies its selection. This implementation activates only one path per LLM call.

For our specific use case, selecting a model for a task follows these steps:

1. Router LLM receives user prompt and route information, outputs JSON with `model_choice` and `reason`
2. Simple LLM call executes `model_choice` and returns response

Now let's see the coded implementation of this workflow.

## Setup and Utils

In [3]:
# Install libraries
!pip install -qU pydantic together

In [4]:
from google.colab import userdata
userdata.get('secretName')

SecretNotFoundError: Secret secretName does not exist.

In [ ]:
# Import libraries
import json
import together
from together import Together

from typing import Any, Optional, Dict, List, Literal
from pydantic import Field, BaseModel, ValidationError

TOGETHER_API_KEY = "--Your API Key--"

client = Together(api_key= TOGETHER_API_KEY)

In [ ]:
# Simple LLM call helper function
def run_llm(user_prompt : str, model : str, system_prompt : Optional[str] = None):
    """ Run the language model with the given user prompt and system prompt. """
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": user_prompt})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=4000,
    )

    return response.choices[0].message.content

# Simple JSON mode LLM call helper function
def JSON_llm(user_prompt : str, schema : BaseModel, system_prompt : Optional[str] = None):
    """ Run a language model with the given user prompt and system prompt, and return a structured JSON object. """
    try:
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})

        messages.append({"role": "user", "content": user_prompt})

        extract = client.chat.completions.create(
            messages=messages,
            model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            response_format={
                "type": "json_object",
                "schema": schema.model_json_schema(),
            },
        )

        response = json.loads(extract.choices[0].message.content)
        return response

    except ValidationError as e:
        raise ValueError(f"Schema validation failed: {str(e)}")

## Routing Agent Implementation

The overall flow of what we need to implement will be as follows:

#### Router LLM

The router LLM will need to select a route - we will create a Pydantic class to capture all available routes. This class will serve as both a guide for the Router LLM's output format and enable parsing its response to execute the chosen model.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class ModelOutput(BaseModel):
    model: Literal["deepseek-ai/DeepSeek-V3",  # All model choices that can be selected
                   "Qwen/Qwen2.5-Coder-32B-Instruct",
                   "Gryphe/MythoMax-L2-13b",
                   "Qwen/QwQ-32B-Preview",
                   "meta-llama/Llama-3.3-70B-Instruct-Turbo"]

    reason: str = Field(   # We need the router to tell us why the model/route was selected
        description="Reason why this model was selected for the task specified in the prompt/query."
    )

In [ ]:
# We will use the pydantic class to both structure the output prompt and to give the router LLM information about routes
# This will help the router make a better decision on which model to select

ROUTER_SYSTEM_PROMPT = """Given a user prompt/query, select the best model from the available options to solve the task and provide a reason for your choice.
Each model has different capabilities - select the best model for the task provided:
- deepseek-ai/DeepSeek-V3: Good generic model to default to incase no better alternative is selected
- Qwen/Qwen2.5-Coder-32B-Instruct: Best for code generation tasks
- Gryphe/MythoMax-L2-13b: Best model for story-telling and role-play and fantasy tasks
- Qwen/QwQ-32B-Preview: Best model for reasoning, math and muiltistep tasks
- meta-llama/Llama-3.3-70B-Instruct-Turbo: Best model for general enterprise usecases and tasks"""

ROUTER_PROMPT = "Given a user prompt/query: {user_query}, select the best model from the available options to solve the task and provide a reason for your choice. Answer only in JSON format."

prompt = "Produce python code snippet to check to see if a number is prime or not."

selected_model = JSON_llm(ROUTER_PROMPT.format(user_query=prompt),
                            ModelOutput,
                            system_prompt=ROUTER_SYSTEM_PROMPT)

selected_model

Lets create a small function that we can execute for multiple tasks

In [ ]:
# Write a function that will call the router and then the output llm model in sequence to generate a response to a user prompt.
def run_router_workflow(user_prompt : str):

    # Which route to take
    selected_model = JSON_llm(ROUTER_PROMPT.format(user_query=user_prompt),
                            ModelOutput,
                            system_prompt=ROUTER_SYSTEM_PROMPT)

    # Take that route and run the LLM model
    response = run_llm(user_prompt= user_prompt,
                   model = selected_model['model']
    )
    return selected_model['model'], selected_model['reason'], response

In [ ]:
model, reason, response = run_router_workflow(prompt)

print(f"Query: {prompt}")
print(20*'==')
print(f"Selected Model: {model} \n Reason: {reason}")
print(20*'==')
print(f"Response: {response}")

### Generic Implementation

Now that we know how the internals of this workflow execute we can write a more generic version.

In [ ]:
def router_workflow(input_query: str, routes : Dict[str, str]) -> str:
    """ Given a `input_qeury` and a dictionary of `routes` containing options and details for each.
    Selects the best model for the task and return the response from the model.
    """
    ROUTER_PROMPT = """Given a user prompt/query: {user_query}, select the best option out of the following routes:
    {routes}. Answer only in JSON format."""

    # Create a schema from the routes dictionary
    class Schema(BaseModel):
        route: Literal[tuple(routes.keys())]

        reason: str = Field(
            description="Short one-liner explanation why this route was selected for the task in the prompt/query."
        )

    # Call LLM to select route
    selected_route = JSON_llm(ROUTER_PROMPT.format(user_query=input_query, routes=routes), Schema)
    print(f"Selected route:{selected_route['route']}\nReason: {selected_route['reason']}\n")

    # Use LLM on selected route.
    # Could also have different prompts that need to be used for each route.
    response = run_llm(user_prompt= input_query, model = selected_route['route'])
    print(f"Response: {response}\n")

    return response

In [ ]:
# Example usage

prompt_list = ["Produce python snippet to check to see if a number is prime or not.",
               "Plan and provide a short itenary for a 2 week vacation in Europe.",
               "Write a short story about a dragon and a knight."]

model_routes = { # feel free to add more models and their descriptions
    "Qwen/Qwen2.5-Coder-32B-Instruct" : "Best model choice for code generation tasks.",
    "Gryphe/MythoMax-L2-13b" : "Best model choice for story-telling, role-playing and fantasy tasks.",
    "Qwen/QwQ-32B-Preview" : "Best model for reasoning, planning and muilti-step tasks",
}

for i, prompt in enumerate(prompt_list):
    print(f"Task {i+1}: {prompt}\n")
    print(20*'==')
    router_workflow(prompt, model_routes)
